In [30]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

# Set device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the CNN model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=3):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)

        # Calculate the size of the input to the fully connected layer dynamically
        self.fc_input_size = 32 * (128 // 2**2) * (128 // 2**2)
        
        self.fc1 = nn.Linear(self.fc_input_size, num_classes)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, self.fc_input_size)  # Flatten before FC layer
        x = self.fc1(x)
        return x

# Define data transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# Load dataset
dataset = datasets.ImageFolder(root='data', transform=transform)

# Define the size of the training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# Split the dataset into training and validation sets
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

# Initialize the model, loss function, and optimizer
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 12

for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate accuracy
    accuracy = correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Validation Accuracy: {accuracy:.2%}')

# Save the trained model
torch.save(model.state_dict(), 'simple_cnn_model.pth')


Epoch [1/12], Loss: 0.8707, Validation Accuracy: 57.54%
Epoch [2/12], Loss: 0.6396, Validation Accuracy: 67.60%
Epoch [3/12], Loss: 0.9665, Validation Accuracy: 79.89%
Epoch [4/12], Loss: 0.4291, Validation Accuracy: 79.89%
Epoch [5/12], Loss: 0.5941, Validation Accuracy: 81.56%
Epoch [6/12], Loss: 0.5459, Validation Accuracy: 80.45%
Epoch [7/12], Loss: 0.3083, Validation Accuracy: 76.54%
Epoch [8/12], Loss: 0.2392, Validation Accuracy: 82.12%
Epoch [9/12], Loss: 0.3320, Validation Accuracy: 82.68%
Epoch [10/12], Loss: 0.4333, Validation Accuracy: 80.45%
Epoch [11/12], Loss: 0.3683, Validation Accuracy: 79.89%
Epoch [12/12], Loss: 0.2845, Validation Accuracy: 81.56%


In [38]:
from PIL import Image
import os

def crop_images(input_folder, output_folder, crop_box):
    # Ensure output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate through all files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            # Open the image
            image_path = os.path.join(input_folder, filename)
            img = Image.open(image_path)

            # Crop the image
            cropped_img = img.crop(crop_box)

            # Save the cropped image to the output folder
            output_path = os.path.join(output_folder, filename)
            cropped_img.save(output_path)

if __name__ == "__main__":
    # Specify your input and output folders
    input_folder = "/Users/yashahuja/Downloads/content/test1"
    output_folder = "/Users/yashahuja/Downloads/content/test2"

    # Specify the crop box (left, upper, right, lower)
    crop_box = (84,24,1157,339)

    # Call the function to crop images
    crop_images(input_folder, output_folder, crop_box)


In [47]:
import torch
from torchvision import transforms
from PIL import Image

# Load the trained model
model = SimpleCNN()
model.load_state_dict(torch.load('simple_cnn_model.pth'))
model.eval()

# Preprocess the input image
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
    ])
    image = Image.open(image_path)

    # If the image has an alpha channel, remove it
    if image.mode == 'RGBA':
        image = image.convert('RGB')

    image = transform(image).unsqueeze(0)  # Add batch dimension
    return image
# Make predictions
def predict_image(model, image_path):
    model.eval()
    image = preprocess_image(image_path)
    with torch.no_grad():
        output = model(image)
        _, predicted_class = torch.max(output, 1)
        return predicted_class.item()

# Example usage
image_path = 'test2/201310181200_rx3_30min51bin.png'
predicted_class = predict_image(model, image_path)
dic1 = {0: "Layer is found", 1: "No Layer", 2: "Data Missing"}
print(f'Result: {dic1[predicted_class]}')


Result: Layer is found
